In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn 
from sklearn.externals import joblib
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import pickle

In [4]:
df = pd.read_csv("ds.csv")
df.head()

,Contour Area,Mean,Blinking,Voice,Music
0,NaN,NaN,NaN,NaN,NaN
1,335.0,NaN,0.0,0.0,5.0
2,NaN,NaN,NaN,NaN,NaN
3,335.0,73.08525,1.0,0.0,3.0
4,NaN,NaN,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4088 entries, 0 to 4087
Data columns (total 5 columns):
Contour Area    2044 non-null float64
Mean            2043 non-null float64
Blinking        2044 non-null float64
Voice           2044 non-null float64
Music           2044 non-null float64
dtypes: float64(5)
memory usage: 159.8 KB


In [8]:
data = df.dropna()
data = data.reset_index(drop=True)
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2043 entries, 0 to 2042
Data columns (total 5 columns):
Contour Area    2043 non-null float64
Mean            2043 non-null float64
Blinking        2043 non-null float64
Voice           2043 non-null float64
Music           2043 non-null float64
dtypes: float64(5)
memory usage: 79.9 KB


,Contour Area,Mean,Blinking,Voice,Music
count,2043.000000,2043.000000,2043.000000,2043.000000,2043.000000
mean,347.994126,27.556291,0.496329,0.511992,4.931473
std,33.568497,6.163387,0.500109,0.499979,3.168807
min,304.000000,24.362080,0.000000,0.000000,0.000000
25%,335.000000,24.904857,0.000000,0.000000,2.000000
50%,335.000000,26.162346,0.000000,1.000000,5.000000
75%,370.000000,28.784579,1.000000,1.000000,8.000000
max,399.000000,118.064270,1.000000,1.000000,10.000000


In [9]:
input_cols = ['Contour Area','Mean','Blinking','Voice']
out_cols = ['Music']
X = data[input_cols]
y = data[out_cols]
print( X.shape, y.shape)

(2043, 4) (2043, 1)


In [10]:
split = int(0.9 * data.shape[0])
training_data = data[:split]
testing_data = data[split:]

In [17]:
rf = RandomForestClassifier(n_estimators=200, criterion='entropy')
rf.fit(X[:split], y[:split])
preds = rf.predict(X[split:])
rf.score(X[split:], y[split:])
rmse = np.sqrt(mean_squared_error(y[split:], preds))
print(rmse)

/home/subtleseeker/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


4.334208792116335


In [14]:
data_dmatrix = xgb.DMatrix(data=X, label=y)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.head()

,Contour Area,Mean,Blinking,Voice
1209,304.0,24.712503,0.0,0.0
629,335.0,29.015892,0.0,0.0
1332,335.0,24.591759,1.0,1.0
1288,335.0,25.049100,1.0,1.0
307,370.0,27.133440,0.0,0.0


In [18]:
xg_reg = xgb.XGBClassifier(objective='multi:softprob')

In [19]:
eval_set = [(X_test, y_test)]

In [20]:
xg_reg.fit(X_train,y_train,early_stopping_rounds=10,eval_metric=["mlogloss"], eval_set=eval_set, verbose=True)
preds = xg_reg.predict(X_test)
yt = np.asarray(y_test)
# pred = xg_reg.predict(X_train)
# rmse = np.sqrt(mean_squared_error(y_train, pred))
# print("RMSE: %f" % (rmse))
# print(xg_reg.score(X_train,y_train))

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
#print(preds)
#yt1 = np.reshape(yt,(409,))
#acc = 100.0 *(preds == yt1).sum()/preds.shape
print(xg_reg.score(X_test,y_test))


[0]	validation_0-mlogloss:2.39833
Will train until validation_0-mlogloss hasn't improved in 10 rounds.
[1]	validation_0-mlogloss:2.3986
[2]	validation_0-mlogloss:2.39963
[3]	validation_0-mlogloss:2.40136
[4]	validation_0-mlogloss:2.40189
[5]	validation_0-mlogloss:2.40358
[6]	validation_0-mlogloss:2.40586
[7]	validation_0-mlogloss:2.40725
[8]	validation_0-mlogloss:2.40987
[9]	validation_0-mlogloss:2.4117
[10]	validation_0-mlogloss:2.4147
Stopping. Best iteration:
[0]	validation_0-mlogloss:2.39833

RMSE: 4.415465
0.10024449877750612


/home/subtleseeker/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/subtleseeker/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/subtleseeker/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
# filename = 'finalized_model.sav'
# pickle.dump(xg_reg, open(filename, 'wb'))

filename = 'finalized_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [24]:
l = [[355.0,26.7,1.0,1.0]]
#df = pd.DataFrame({'Contour Area':l[:1]})#result = (loaded_model.predict([355.0,27.77,1.0,1.0]))
new_df = pd.DataFrame(columns=['Contour Area', 'Mean','Blinking','Voice'], data=l)
result = loaded_model.predict(new_df)
print(result)

[5.]
